In [1]:
import pandas as pd
from api.core.mongo import db
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

In [2]:
rows = pd.DataFrame(list(db["products"].find({}))).drop(columns=["_id"]).to_dict(orient="records")

In [3]:
splitter = RecursiveJsonSplitter(max_chunk_size=300)
docs = splitter.create_documents(texts=rows)

In [5]:
# Load the embedding model (https://huggingface.co/nomic-ai/nomic-embed-text-v1")
model = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={ "trust_remote_code": True })

<All keys matched successfully>


In [6]:
# Store the data as vector embeddings in Atlas
vector_store = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = model,
    collection = "embeddings",
    index_name = "vector_index"
)

RuntimeError: MPS backend out of memory (MPS allocated: 6.30 GB, other allocations: 861.88 MB, max allowed: 9.07 GB). Tried to allocate 2.25 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).